# 002 - Simple Neural Network


Following the example in https://pytorch.org/tutorials/advanced/cpp_frontend.html


In [4]:
#include<iostream>
#include<string>
std::cerr << "Kernel initialized\n";

// Setup: TODO simplify this
// add headers
#pragma cling add_include_path("../libs/libtorch/include/torch/csrc/api/include")
#pragma cling add_include_path("../libs/libtorch/include/")
// add precompiled libtorch*.so files
#pragma cling add_library_path("../libs/libtorch/lib/")
#pragma cling load("libtorch.so")

//// --- setup is ready -- time to import torch
#include <torch/torch.h>
std::cerr << "torch initialized";

Kernel initialized
torch initialized

## Simple network


Reference: https://pytorch.org/cppdocs/frontend.html#end-to-end-example


In [9]:
!bash ../data/setup.sh  to download data

/mnt/home/tg/work/repos/me/libtorch-tutorial/data/MNIST/raw/._OK exists. skipping


In [10]:
auto mnist_data_dir = "../data/MNIST/raw";

// Define a new Module.
struct Net : torch::nn::Module {
  Net() {
    // Construct and register two Linear submodules.
    fc1 = register_module("fc1", torch::nn::Linear(784, 64));
    fc2 = register_module("fc2", torch::nn::Linear(64, 32));
    fc3 = register_module("fc3", torch::nn::Linear(32, 10));
  }

  // Implement the Net's algorithm.
  torch::Tensor forward(torch::Tensor x) {
    // Use one of many tensor manipulation functions.
    x = torch::relu(fc1->forward(x.reshape({x.size(0), 784})));
    x = torch::dropout(x, /*p=*/0.5, /*train=*/is_training());
    x = torch::relu(fc2->forward(x));
    x = torch::log_softmax(fc3->forward(x), /*dim=*/1);
    return x;
  }

  // Use one of many "standard library" modules.
  torch::nn::Linear fc1{nullptr}, fc2{nullptr}, fc3{nullptr};
};


// Create a new Net.
auto net = std::make_shared<Net>();

// Create a multi-threaded data loader for the MNIST dataset.
auto data_loader = torch::data::make_data_loader(
  torch::data::datasets::MNIST(mnist_data_dir).map(
      torch::data::transforms::Stack<>()),
  /*batch_size=*/64);

// Instantiate an SGD optimization algorithm to update our Net's parameters.
torch::optim::SGD optimizer(net->parameters(), /*lr=*/0.01);

for (size_t epoch = 1; epoch <= 10; ++epoch) {
    size_t batch_index = 0;
    // Iterate the data loader to yield batches from the dataset.
    for (auto& batch : *data_loader) {
      // Reset gradients.
      optimizer.zero_grad();
      // Execute the model on the input data.
      torch::Tensor prediction = net->forward(batch.data);
      // Compute a loss value to judge the prediction of our model.
      torch::Tensor loss = torch::nll_loss(prediction, batch.target);
      // Compute gradients of the loss w.r.t. the parameters of our model.
      loss.backward();
      // Update the parameters based on the calculated gradients.
      optimizer.step();
      // Output the loss and checkpoint every 100 batches.
      if (++batch_index % 100 == 0) {
        std::cout << "Epoch: " << epoch << " | Batch: " << batch_index
                  << " | Loss: " << loss.item<float>() << std::endl;
        // Serialize your model periodically as a checkpoint.
        torch::save(net, "net.pt");
      }
    }
}


Epoch: 1 | Batch: 100 | Loss: 2.29083
Epoch: 1 | Batch: 200 | Loss: 2.17135
Epoch: 1 | Batch: 300 | Loss: 2.21064
Epoch: 1 | Batch: 400 | Loss: 2.06644
Epoch: 1 | Batch: 500 | Loss: 1.89153
Epoch: 1 | Batch: 600 | Loss: 1.92552
Epoch: 1 | Batch: 700 | Loss: 1.74535
Epoch: 1 | Batch: 800 | Loss: 1.40254
Epoch: 1 | Batch: 900 | Loss: 1.31521
Epoch: 2 | Batch: 100 | Loss: 1.0744
Epoch: 2 | Batch: 200 | Loss: 1.08279
Epoch: 2 | Batch: 300 | Loss: 1.16592
Epoch: 2 | Batch: 400 | Loss: 0.836995
Epoch: 2 | Batch: 500 | Loss: 0.938627
Epoch: 2 | Batch: 600 | Loss: 1.11453
Epoch: 2 | Batch: 700 | Loss: 0.696703
Epoch: 2 | Batch: 800 | Loss: 0.69518
Epoch: 2 | Batch: 900 | Loss: 0.640828
Epoch: 3 | Batch: 100 | Loss: 0.905752
Epoch: 3 | Batch: 200 | Loss: 0.755499
Epoch: 3 | Batch: 300 | Loss: 0.809715
Epoch: 3 | Batch: 400 | Loss: 0.730176
Epoch: 3 | Batch: 500 | Loss: 0.756368
Epoch: 3 | Batch: 600 | Loss: 0.70157
Epoch: 3 | Batch: 700 | Loss: 0.739328
Epoch: 3 | Batch: 800 | Loss: 0.720477
Ep